# Classify COVID-19 datasets based on routine blood examination

Read and understand the paper “Development, evaluation, and validation of machine learning models
for COVID-19 detection based on routine blood tests”. The research paper and the dataset can be
found in the attached folder named “SARS-CoV-2”.

Try to replicate the ML models as described in the research paper by applying all steps the paper
introduces to you and document which of their steps you had to take an educated guess for.

Additionally, explore how feature optimization can improve your results. Accuracy, AOC and other
benchmarking criteria to compare your models with the researcher’s published ones should be
displayed just as in the table of the paper.

# Steps for ML Model Building
1. Discard all features with a missing data rate greater than 75%
2. Data imputation using multivariate k-nearest neighbour algorithm with k = 5
3. Feature selection using recursive feature-elimination algorithm, optimal features selected through hyper-parameter optimization
4. Evaluate classification algorithms:
    - Random Forest
    - Naive Bayes 
    - Logistic Regression
    - Support Vector Machine
    - k-Nearest Neighbours
5. Tune hyperparameters using grid search approach


## For Model Selection
1. Split data set into training set (80%) and test set (20%) using stratified approach
2. Hyper-parameter optimization using 5-fold stratified cross-validation grid search using AUC as reference measure
3. Train and calibrate on entire training set
4. Evaluate on test set using measures of accuracy, sensitivity, specificity, AUC and Brier score
5. For each model class, consider the standard version and a 3-way version (model that abstains from predictions where confidence score is below 75%)

Additionally, control randomisation in all stages of model development to ensure repeatability.

Libraries utilised:
- numpy
- pandas
- scikit-learn


## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Data Pre-Processing

In [4]:
dataset = pd.read_csv(r'C:\Users\Cindy\Downloads\Trial Task Material Extracted\Resources (PIT-Trial Task)\SARS-CoV-2\all_training.csv')
is_altered = False
print(dataset.shape)
dataset

(1736, 36)


Unnamed: 0  Sex   Age    CA     CK   CREA    ALP    GGT    GLU  \
0     A00345_2020-03-25  1.0  82.0  2.09    NaN  1.150   95.0   40.0   78.0   
1     A00791_2020-03-19  1.0  51.0  1.97  237.0  0.970   54.0   98.0   98.0   
2     A00741_2020-03-04  1.0  58.0  2.11    NaN  1.000   80.0  147.0  106.0   
3     A00605_2020-04-15  0.0  82.0  2.27  138.0  0.755  123.5  176.5  106.0   
4     A00417_2020-02-24  1.0  79.0  2.07   73.0  1.810   62.0   36.5   96.0   
...                 ...  ...   ...   ...    ...    ...    ...    ...    ...   
1731                 49  0.0   NaN  2.38   40.0  0.800   68.0    9.0  128.0   
1732                 50  0.0   NaN  2.36    NaN  0.960   79.0   35.0  107.0   
1733                 51  1.0   NaN  2.28    NaN  1.420    NaN    NaN  136.0   
1734                 52  1.0   NaN  2.40  124.0  0.950   48.0   44.0   95.0   
1735                 53  1.0   NaN  2.14    NaN  1.040    NaN    NaN  179.0   

        AST  ...    MO   EO   BA   NET   LYT  MOT  EOT  BAT  Suspect  target  
0      26.0  ...   9.5  2.9  0.5  6.40  1.20  0.8  0.3  0.0      1.0       0  
1      74.0  ...   NaN  NaN  NaN   NaN   NaN  NaN  NaN  NaN      1.0       1  
2      41.0  ...   7.3  0.3  0.1  5.45  0.75  0.5  0.0  0.0      1.0       0  
3     114.0  ...   9.5  1.7  0.9  3.60  2.60  0.7  0.1  0.1      0.5       0  
4      28.0  ...  10.0  8.5  0.5  0.40  0.50  0.1  0.1  0.0      1.0       0  
...     ...  ...   ...  ...  ...   ...   ...  ...  ...  ...      ...     ...  
1731   13.0  ...   NaN  NaN  NaN   NaN   NaN  NaN  NaN  NaN      1.0       0  
1732   24.0  ...   NaN  NaN  NaN   NaN   NaN  NaN  NaN  NaN      1.0       0  
1733   53.0  ...   NaN  NaN  NaN   NaN   NaN  NaN  NaN  NaN      1.0       0  
1734   50.0  ...   NaN  NaN  NaN   NaN   NaN  NaN  NaN  NaN      1.0       0  
1735   28.0  ...   NaN  NaN  NaN   NaN   NaN  NaN  NaN  NaN      1.0       0  

[1736 rows x 36 columns]

In [5]:
dataset = dataset.dropna(axis = 1, thresh = int(0.75 * dataset.shape[0]))
print(dataset.shape)

(1736, 32)


In [6]:
if type(dataset) is not np.ndarray:
    dataset = dataset.to_numpy()
    

if is_altered is False:
    dataset = dataset[:,1:]
    dataset
    is_altered = True
    
dataset

array([[1.0, 82.0, 2.09, ..., 0.0, 1.0, 0],
       [1.0, 51.0, 1.97, ..., nan, 1.0, 1],
       [1.0, 58.0, 2.11, ..., 0.0, 1.0, 0],
       ...,
       [1.0, nan, 2.28, ..., nan, 1.0, 0],
       [1.0, nan, 2.4, ..., nan, 1.0, 0],
       [1.0, nan, 2.14, ..., nan, 1.0, 0]], dtype=object)

### Splitting Training and Test Data with Straitification

In [7]:
inputs = dataset[:,:-1]
targets = dataset[:,-1]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.2, random_state = 123, stratify = targets)

y_train = y_train.astype('int')
y_test = y_test.astype('int')

# Data Imputation using KNN

In [9]:
from sklearn.impute import KNNImputer

In [10]:
print(X_train)

imputer = KNNImputer(n_neighbors = 5, weights = "uniform")

X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

print(y_train)
X_train

[[0.0 11.0 2.365 ... nan nan 0.0]
 [0.0 60.0 2.22 ... 0.3 0.0 0.0]
 [1.0 47.0 nan ... nan nan 0.0]
 ...
 [1.0 46.0 2.28 ... nan nan 0.5]
 [1.0 86.0 2.19 ... 0.0 0.0 1.0]
 [1.0 76.0 2.0022375 ... 0.15 0.016 1.0]]
[0 0 0 ... 0 1 1]


array([[0.0000000e+00, 1.1000000e+01, 2.3650000e+00, ..., 1.3200000e-01,
        6.0000000e-03, 0.0000000e+00],
       [0.0000000e+00, 6.0000000e+01, 2.2200000e+00, ..., 3.0000000e-01,
        0.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 4.7000000e+01, 2.4140000e+00, ..., 1.0000000e-02,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [1.0000000e+00, 4.6000000e+01, 2.2800000e+00, ..., 2.6000000e-01,
        0.0000000e+00, 5.0000000e-01],
       [1.0000000e+00, 8.6000000e+01, 2.1900000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 7.6000000e+01, 2.0022375e+00, ..., 1.5000000e-01,
        1.6000000e-02, 1.0000000e+00]])

# Recursive Feature Elimination

In [11]:
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

from numpy import mean
from numpy import std

### Hyperparameter selection

In [12]:
def hyperparam_selection(min_range, max_range):    
    optimal_feature_num = 30
    max_accuracy = 0

    for num_features in range(min_range, max_range + 1):
        rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select = num_features)
        model = DecisionTreeClassifier()
        pipeline = Pipeline(steps=[('s',rfe),('m',model)])
        X_train_copy = rfe.fit_transform(X_train, y_train)

        #evaluation
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
        n_scores = cross_val_score(pipeline, X_test, y_test, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        print('Number of features: %d   Accuracy: %.3f (%.3f)' % (num_features, mean(n_scores), std(n_scores)))

        if mean(n_scores) > max_accuracy:
            max_accuracy = mean(n_scores)
            optimal_feature_num = num_features
    
    return optimal_feature_num

In [13]:
opt_feature_num = hyperparam_selection(1, 30)

Number of features: 1   Accuracy: 0.591 (0.094)
Number of features: 2   Accuracy: 0.677 (0.075)
Number of features: 3   Accuracy: 0.708 (0.082)
Number of features: 4   Accuracy: 0.729 (0.072)
Number of features: 5   Accuracy: 0.716 (0.077)
Number of features: 6   Accuracy: 0.720 (0.064)
Number of features: 7   Accuracy: 0.718 (0.064)
Number of features: 8   Accuracy: 0.737 (0.064)
Number of features: 9   Accuracy: 0.746 (0.066)
Number of features: 10   Accuracy: 0.752 (0.054)
Number of features: 11   Accuracy: 0.741 (0.065)
Number of features: 12   Accuracy: 0.742 (0.080)
Number of features: 13   Accuracy: 0.750 (0.062)
Number of features: 14   Accuracy: 0.749 (0.076)
Number of features: 15   Accuracy: 0.742 (0.073)
Number of features: 16   Accuracy: 0.741 (0.072)
Number of features: 17   Accuracy: 0.742 (0.063)
Number of features: 18   Accuracy: 0.743 (0.072)
Number of features: 19   Accuracy: 0.736 (0.076)
Number of features: 20   Accuracy: 0.737 (0.071)
Number of features: 21   Accu

In [14]:
print(opt_feature_num)
print(X_train.shape)
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select = opt_feature_num)
X_train = rfe.fit_transform(X_train, y_train)
X_test = rfe.transform(X_test)

21
(1388, 30)


In [15]:
print(X_train.shape)
print(X_test.shape)

(1388, 21)
(348, 21)


### Feature Normalisation

In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
print(X_test)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_test

[[4.74000000e+01 1.84000000e+00 1.21000000e+00 ... 5.13333334e-02
  3.55555560e-03 1.00000000e+00]
 [3.20000000e+01 2.05000000e+00 8.30000000e-01 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [5.60000000e+01 2.32000000e+00 9.80000000e-01 ... 1.00000000e-01
  1.00000000e-01 5.00000000e-01]
 ...
 [4.70000000e+01 2.48000000e+00 5.30000000e-01 ... 3.00000000e-01
  1.00000000e-01 1.00000000e+00]
 [7.70000000e+01 2.08332500e+00 7.70000000e-01 ... 4.00000000e-02
  6.00000000e-03 0.00000000e+00]
 [7.30000000e+01 2.17896667e+00 5.70000000e-01 ... 1.06666667e-01
  3.33333330e-02 1.00000000e+00]]


array([[-0.67640729, -2.18664511,  0.06475289, ..., -0.14971628,
        -0.34798244,  0.63517156],
       [-1.45985134, -0.93814126, -0.34414052, ..., -0.55905691,
        -0.44518681,  0.63517156],
       [-0.23889958,  0.66707797, -0.18273523, ...,  0.2383599 ,
         2.28868624, -0.68659474],
       ...,
       [-0.69675649,  1.618319  , -0.66695111, ...,  1.83319353,
         2.28868624,  0.63517156],
       [ 0.82943321, -0.74001559, -0.40870264, ..., -0.24009019,
        -0.28115443, -2.00836104],
       [ 0.62594125, -0.17140136, -0.6239097 , ...,  0.29152102,
         0.4661042 ,  0.63517156]])

# Classification Algorithms Model Creation and Hyperparameter Tuning

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, make_scorer

auc = make_scorer(roc_auc_score)

## Random Forest

In [19]:
RFModel = RandomForestClassifier(random_state = 123)

criterion = ['gini', 'entropy']
max_depth = [1, 3, 5, None]
max_features = ['auto', 'sqrt', 'log2']

grid = GridSearchCV(estimator = RFModel, scoring = auc, param_grid = dict(criterion = criterion, max_depth = max_depth, max_features = max_features))
# default setting already uses the desires 5-fold cross-validation

In [20]:
grid.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(random_state=123),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [1, 3, 5, None],
                         'max_features': ['auto', 'sqrt', 'log2']},
             scoring=make_scorer(roc_auc_score))

In [21]:
print(grid.best_score_)
print(grid.best_params_)

0.8149335382214336
{'criterion': 'gini', 'max_depth': 5, 'max_features': 'auto'}


In [22]:
RFModel = RandomForestClassifier(criterion = 'gini', max_depth = None, max_features = 'auto', random_state = 123)

RFModel.fit(X_train, y_train)

RandomForestClassifier(random_state=123)

## Naive Bayes

In [23]:
NBModel = GaussianNB()

var_smoothing = [1e-10, 3e-10, 1e-9, 3e-9]

grid = GridSearchCV(estimator = NBModel, scoring = auc, param_grid = dict(var_smoothing = var_smoothing))

In [24]:
grid.fit(X_train, y_train)

GridSearchCV(estimator=GaussianNB(),
             param_grid={'var_smoothing': [1e-10, 3e-10, 1e-09, 3e-09]},
             scoring=make_scorer(roc_auc_score))

In [25]:
print(grid.best_score_)
print(grid.best_params_)

0.7795527743401245
{'var_smoothing': 1e-10}


In [26]:
NBModel = GaussianNB(var_smoothing = 1e-10)

NBModel.fit(X_train, y_train)

GaussianNB(var_smoothing=1e-10)

## Logisitic Regression

In [27]:
LRModel = LogisticRegression(random_state = 123)

penalty = ['l1', 'l2', 'elasticnet', 'none']
solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
max_iter = [300, 1000, 2000, 3000]

grid = GridSearchCV(estimator = LRModel, scoring = auc, param_grid = dict(penalty = penalty, solver = solver, max_iter = max_iter))

In [29]:
import warnings
warnings.filterwarnings('ignore')
grid.fit(X_train, y_train)

GridSearchCV(estimator=LogisticRegression(random_state=123),
             param_grid={'max_iter': [300, 1000, 2000, 3000],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring=make_scorer(roc_auc_score))

In [30]:
print(grid.best_score_)
print(grid.best_params_)

0.808791566681425
{'max_iter': 300, 'penalty': 'l1', 'solver': 'saga'}


In [32]:
LRModel = LogisticRegression(penalty = 'l1', solver = 'saga', max_iter = 300, random_state = 123)

LRModel.fit(X_train, y_train)

LogisticRegression(max_iter=300, penalty='l1', random_state=123, solver='saga')

## Support Vector Machine

In [33]:
SVMModel = SVC(random_state = 123)

C = [0.1, 0.3, 1, 3]
degree = [1, 2, 3, 4]
kernel = ['linear', 'poly', 'rbf', 'sigmoid']

grid = GridSearchCV(estimator = SVMModel, scoring = auc, param_grid = dict(C = C, degree = degree, kernel = kernel))
# default setting already uses the desires 5-fold cross-validation

In [34]:
grid.fit(X_train, y_train)

GridSearchCV(estimator=SVC(random_state=123),
             param_grid={'C': [0.1, 0.3, 1, 3], 'degree': [1, 2, 3, 4],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring=make_scorer(roc_auc_score))

In [35]:
print(grid.best_score_)
print(grid.best_params_)

0.8164708548350861
{'C': 0.3, 'degree': 1, 'kernel': 'linear'}


In [39]:
SVMModel = SVC(C = 0.3, degree = 1, kernel = 'linear', random_state = 123, probability = True)

SVMModel.fit(X_train, y_train)

SVC(C=0.3, degree=1, kernel='linear', probability=True, random_state=123)

## K-nearest Neighbours

In [40]:
KNNModel = KNeighborsClassifier()

n_neighbors = [3, 5, 10, 30, 100]

grid = GridSearchCV(estimator = KNNModel, scoring = auc, param_grid = dict(n_neighbors = n_neighbors))

In [41]:
grid.fit(X_train, y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 10, 30, 100]},
             scoring=make_scorer(roc_auc_score))

In [42]:
print(grid.best_score_)
print(grid.best_params_)

0.799089334040261
{'n_neighbors': 30}


In [44]:
KNNModel = KNeighborsClassifier(n_neighbors = 30)

KNNModel.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=30)

# Model Evaluation Metrics on Test Set

Evaluation based on Metrics of:
- accuracy
- sensitivity 
- specificity 
- AUC
- Brier score

In [45]:
from sklearn.metrics import accuracy_score, roc_auc_score, brier_score_loss, confusion_matrix

### Function that outputs all evaluation metrics on test set

In [51]:
def eval_metrics(typeModel, _X_test, _y_test):
    
    y_preds = typeModel.predict(X_test)
    
    accuracy = accuracy_score(_y_test, y_preds)    
    AUC = roc_auc_score(_y_test, y_preds)    
    brier_score = brier_score_loss(_y_test, y_preds)
    
    tn, fp, fn, tp = confusion_matrix(_y_test, y_preds).ravel()
    
    sensitivity = tp / (tp + fn)    
    specificity = tn / (tn + fp)
        
    return accuracy, sensitivity, specificity, AUC, brier_score

### Function that outputs all evaluation metrics on test set only when model has confidence of >75%

In [52]:
def eval_metrics_3_way(typeModel, _X_test, _y_test):
    
    y_pred_prob = typeModel.predict_proba(_X_test)

    y_max_pred_prob = np.amax(y_pred_prob, 1)

    y_pred = typeModel.predict(X_test)

    y_max_pred_prob = np.expand_dims(y_max_pred_prob, axis = 1)
    y_pred = np.expand_dims(y_pred, axis = 1)
    y_test_copy = np.expand_dims(_y_test, axis = 1)

    confidence_array = np.concatenate((y_max_pred_prob, y_pred, y_test_copy),axis = 1)

    culled_confidence_array = confidence_array[confidence_array[:,0] > 0.75]

    confident_y_preds = culled_confidence_array[:,1]
    
    confident_y_test = culled_confidence_array[:,2]
    
    accuracy = accuracy_score(confident_y_test, confident_y_preds)    
    AUC = roc_auc_score(confident_y_test, confident_y_preds)    
    brier_score = brier_score_loss(confident_y_test, confident_y_preds)
    
    tn, fp, fn, tp = confusion_matrix(confident_y_test, confident_y_preds).ravel()
    
    sensitivity = tp / (tp + fn)    
    specificity = tn / (tn + fp)
    
    coverage = confident_y_preds.shape[0] / y_pred.shape[0]
        
    return accuracy, sensitivity, specificity, AUC, brier_score, coverage

## Random Forest

In [53]:
RF_accuracy, RF_sensitivity, RF_specificity, RF_AUC, RF_brier_score = eval_metrics(RFModel, X_test, y_test)

print(RF_accuracy, RF_sensitivity, RF_specificity, RF_AUC, RF_brier_score)



0.8247126436781609 0.7804878048780488 0.8641304347826086 0.8223091198303287 0.1752873563218391


In [55]:
RF_accuracy_3_way, RF_sensitivity_3_way, RF_specificity_3_way, RF_AUC_3_way, RF_brier_score_3_way, RF_coverage = eval_metrics_3_way(RFModel, X_test, y_test)

print(RF_accuracy_3_way, RF_sensitivity_3_way, RF_specificity_3_way, RF_AUC_3_way, RF_brier_score_3_way, RF_coverage)

0.9195979899497487 0.8681318681318682 0.9629629629629629 0.9155474155474156 0.08040201005025126 0.5718390804597702


## Naive Bayes

In [56]:
NB_accuracy, NB_sensitivity, NB_specificity, NB_AUC, NB_brier_score = eval_metrics(NBModel, X_test, y_test)

print(NB_accuracy, NB_sensitivity, NB_specificity, NB_AUC, NB_brier_score)

0.7701149425287356 0.8109756097560976 0.7336956521739131 0.7723356309650053 0.22988505747126436


In [57]:
NB_accuracy_3_way, NB_sensitivity_3_way, NB_specificity_3_way, NB_AUC_3_way, NB_brier_score_3_way, NB_coverage = eval_metrics_3_way(NBModel, X_test, y_test)

print(NB_accuracy_3_way, NB_sensitivity_3_way, NB_specificity_3_way, NB_AUC_3_way, NB_brier_score_3_way, NB_coverage)

0.7940199335548173 0.8461538461538461 0.7468354430379747 0.7964946445959105 0.2059800664451827 0.8649425287356322


## Logisitic Regression

In [58]:
LR_accuracy, LR_sensitivity, LR_specificity, LR_AUC, LR_brier_score = eval_metrics(LRModel, X_test, y_test)

print(LR_accuracy, LR_sensitivity, LR_specificity, LR_AUC, LR_brier_score)

0.8103448275862069 0.7987804878048781 0.8206521739130435 0.8097163308589608 0.1896551724137931


In [59]:
LR_accuracy_3_way, LR_sensitivity_3_way, LR_specificity_3_way, LR_AUC_3_way, LR_brier_score_3_way, LR_coverage = eval_metrics_3_way(LRModel, X_test, y_test)

print(LR_accuracy_3_way, LR_sensitivity_3_way, LR_specificity_3_way, LR_AUC_3_way, LR_brier_score_3_way, LR_coverage)

0.8935185185185185 0.8936170212765957 0.8934426229508197 0.8935298221137077 0.10648148148148148 0.6206896551724138


## Support Vector Machine

In [62]:
SVM_accuracy, SVM_sensitivity, SVM_specificity, SVM_AUC, SVM_brier_score = eval_metrics(SVMModel, X_test, y_test)

print(SVM_accuracy, SVM_sensitivity, SVM_specificity, SVM_AUC, SVM_brier_score)

0.8132183908045977 0.7987804878048781 0.8260869565217391 0.8124337221633086 0.1867816091954023


In [63]:
SVM_accuracy_3_way, SVM_sensitivity_3_way, SVM_specificity_3_way, SVM_AUC_3_way, SVM_brier_score_3_way, SVM_coverage = eval_metrics_3_way(SVMModel, X_test, y_test)

print(SVM_accuracy_3_way, SVM_sensitivity_3_way, SVM_specificity_3_way, SVM_AUC_3_way, SVM_brier_score_3_way, SVM_coverage)

0.915 0.896551724137931 0.9292035398230089 0.9128776319804699 0.085 0.5747126436781609


## k-Nearest Neighbours

In [64]:
KNN_accuracy, KNN_sensitivity, KNN_specificity, KNN_AUC, KNN_brier_score = eval_metrics(KNNModel, X_test, y_test)

print(KNN_accuracy, KNN_sensitivity, KNN_specificity, KNN_AUC, KNN_brier_score)

0.7729885057471264 0.7865853658536586 0.7608695652173914 0.773727465535525 0.22701149425287356


In [66]:
KNN_accuracy_3_way, KNN_sensitivity_3_way, KNN_specificity_3_way, KNN_AUC_3_way, KNN_brier_score_3_way, KNN_coverage = eval_metrics_3_way(KNNModel, X_test, y_test)

print(KNN_accuracy_3_way, KNN_sensitivity_3_way, KNN_specificity_3_way, KNN_AUC_3_way, KNN_brier_score_3_way, KNN_coverage)

0.9064039408866995 0.9021739130434783 0.9099099099099099 0.9060419114766941 0.09359605911330049 0.5833333333333334


In [48]:
#testing code, not part of workflow


y_pred_prob = LRModel.predict_proba(X_test)

y_max_pred_prob = np.amax(y_pred_prob, 1)

y_pred = LRModel.predict(X_test)

y_max_pred_prob = np.expand_dims(y_max_pred_prob, axis = 1)
y_pred = np.expand_dims(y_pred, axis = 1)
y_test_copy = np.expand_dims(y_test, axis = 1)

#print(y_pred_prob.shape)
#print(y_max_pred_prob.shape)
#print((y_pred).shape)
#print(y_test.shape)

confidence_array = np.concatenate((y_max_pred_prob, y_pred, y_test_copy),axis = 1)
#print(confidence_array.shape)
#print(confidence_array[0:20,:])

culled_confidence_array = confidence_array[confidence_array[:,0] > 0.75]
#print(culled_confidence_array.shape)
#print(culled_confidence_array[0:20, :])

confident_y_preds = culled_confidence_array[:,1]

confident_y_test = culled_confidence_array[:,2]
print(confident_y_preds, confident_y_preds.shape[0], y_test.shape[0])

[1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1.
 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0.
 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0.
 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1.
 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0.
 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0.] 216 348
